In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('final_features.csv')

In [3]:
df.head()

,Unnamed: 0,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,...,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,0,0,0,0.833319,0.714276,0.999980,0.999980,0.833326,0.714281,0.0,...,-17.810438,7.231024,1.531186,-7.528823,0.473802,-11.864658,-11.293788,1.866265,3.616046,11.971096
1,1,1,0,0.599988,0.299997,0.499975,0.333322,0.499994,0.307690,0.0,...,23.015827,3.435464,-5.169600,7.102491,34.516881,6.177686,-27.770856,12.926435,-4.564559,33.919834
2,2,2,0,0.333328,0.249997,0.249994,0.166664,0.299997,0.214284,0.0,...,-24.310109,-1.216773,11.909693,9.591573,11.846737,1.397859,6.454157,-0.271460,-12.500337,27.634567
3,3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,-5.435584,1.672591,-0.863278,-2.906553,-3.466688,-3.867892,-4.249463,-12.551012,4.494087,-6.223341
4,4,4,0,0.166664,0.090908,0.999900,0.499975,0.285710,0.153845,0.0,...,-10.407441,-8.444207,-14.450059,-12.709382,-4.449050,12.563987,-11.721362,-16.459300,3.626297,-9.790615


In [4]:
df = df.dropna()

In [5]:
y_true = df['is_duplicate']
df.drop(['Unnamed: 0', 'id', 'is_duplicate'], axis=1, inplace=True)

In [6]:
df.shape

(404287, 626)

## Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(df, y_true, stratify=y_true, test_size=0.3)

## Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [27]:
y_pred = rf.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8365612143098601

In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[70101,  6407],
       [13416, 31363]], dtype=int64)

In [29]:
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.84      0.92      0.88     76508\n           1       0.83      0.70      0.76     44779\n\n    accuracy                           0.84    121287\n   macro avg       0.83      0.81      0.82    121287\nweighted avg       0.84      0.84      0.83    121287\n'

### Hyperparameter of Random Forest using RandomSearch

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [11]:
rf_clf=RandomForestClassifier()

params={
     'n_estimators':[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2500, 3000],
     'max_features' : ['auto', 'sqrt'],
     'max_depth':[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'criterion' : ['gini', 'entropy'],
     'bootstrap': [True, False]
}
random_cfl1=RandomizedSearchCV(rf_clf,param_distributions=params,verbose=10,n_jobs=-1,)
random_cfl1.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
39 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(iterator):
  File 

RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000, 2500,
                                                         3000]},
                   verbose=10)

In [17]:
print (random_cfl1.best_params_)

{'n_estimators': 2000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 50, 'criterion': 'entropy', 'bootstrap': False}


In [8]:
from sklearn.ensemble import RandomForestClassifier
rf_cfl2=RandomForestClassifier(n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_depth=50, max_features='auto', 
                     criterion='entropy', bootstrap=False)

In [9]:
rf_cfl2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=50,
                       min_samples_split=10, n_estimators=2000)

In [10]:
y_pred_tuned = rf_cfl2.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_tuned)

0.848310206370015

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_tuned)

array([[70270,  6238],
       [12160, 32619]], dtype=int64)

In [13]:
import pickle

In [16]:
pickle.dump(rf_cfl2, open('model.pkl','wb'))

## XGBoost

In [31]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (p

[14:41:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
y_pred1 = xgb.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.84326432346418

In [34]:
confusion_matrix(y_test,y_pred1)

array([[68264,  8244],
       [10766, 34013]], dtype=int64)

In [35]:
classification_report(y_test,y_pred1)

'              precision    recall  f1-score   support\n\n           0       0.86      0.89      0.88     76508\n           1       0.80      0.76      0.78     44779\n\n    accuracy                           0.84    121287\n   macro avg       0.83      0.83      0.83    121287\nweighted avg       0.84      0.84      0.84    121287\n'

## Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

C:\Users\arjun\anaconda3\envs\quora\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
y_pred2 = lr.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

0.7745842505792047

In [16]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [15]:
classification_report(y_test,y_pred2)

'              precision    recall  f1-score   support\n\n           0       0.80      0.86      0.83     76508\n           1       0.72      0.63      0.67     44779\n\n    accuracy                           0.77    121287\n   macro avg       0.76      0.74      0.75    121287\nweighted avg       0.77      0.77      0.77    121287\n'

In [17]:
confusion_matrix(y_test,y_pred2)

array([[65737, 10771],
       [16569, 28210]], dtype=int64)

## SVM

In [18]:
from sklearn import svm

In [19]:
svc_clf = svm.SVC()

In [20]:
svc_clf.fit(X_train,y_train)

SVC()

In [22]:
y_pred3 = svc_clf.predict(X_test)

In [23]:

accuracy_score(y_test,y_pred3)

0.8215637290064063

In [25]:
confusion_matrix(y_test,y_pred3)

array([[67773,  8735],
       [12907, 31872]], dtype=int64)

### We have chosen Random Forest Classifier and tuned it further to get our best classifier. 